In [1]:
import xgi
import igraph as ig
import time
import networkx as nx
import matplotlib.pyplot as plt

In [17]:
from line_graph_cpp import compute_line_graph_edges
from tqdm import tqdm

def ultra_fast_line_vector_generator(H, s=1, weights=None):
    if weights not in [None, "absolute", "normalized"]:
        raise ValueError(
            f"{weights} is not a valid weights option. Choices are "
            "None, 'absolute', and 'normalized'."
        )

    LG = ig.Graph(directed=False)
    hyperedges = list(H._edge.items())
    LG.add_vertices(len(hyperedges))
    LG.vs["name"] = [k for k, _ in hyperedges]
    LG.vs["original_hyperedge"] = [v for _, v in hyperedges]

    # Преобразуем гиперрёбра в список множеств
    hyperedge_sets = [set(e[1]) for e in hyperedges]

    # Создаем прогресс-бар
    total_pairs = len(hyperedges) * (len(hyperedges) - 1) // 2
    progress_bar = tqdm(total=total_pairs, desc="Processing pairs")

    def update_progress(step, processed, total):
        progress_bar.update(step)

    # Вызываем C++ функцию
    edges_with_weights = compute_line_graph_edges(
        hyperedge_sets, s, weights if weights else "None", update_progress
    )

    # Закрываем прогресс-бар
    progress_bar.close()

    # Добавляем рёбра и веса
    edges_pairs = [(e[0], e[1]) for e in edges_with_weights]
    LG.add_edges(edges_pairs)
    if weights:
        LG.es["weight"] = [e[2] for e in edges_with_weights]

    return LG

In [18]:
H_enron = xgi.load_xgi_data("ndc-substances")
H_enron_cleaned = H_enron.cleanup(
    multiedges=False, singletons=False, isolates=False, relabel=True, in_place=False
)

In [19]:
from line_graph_cpp import compute_line_graph_edges

# Проверка сигнатуры
help(compute_line_graph_edges)

Help on built-in function compute_line_graph_edges in module line_graph_cpp:

compute_line_graph_edges(...) method of builtins.PyCapsule instance
    compute_line_graph_edges(hyperedges: list[set[int]], s: int, weights: str, progress_callback: Callable) -> list[tuple[int, int, float]]

    Compute line graph edges in C++



In [20]:
#start_time = time.time()

#cent_p = xgi.convert.to_ig_line_graph(H_enron_cleaned)

middle_time = time.time()

cent_cpp = ultra_fast_line_vector_generator(H_enron_cleaned)

end_time = time.time()

#print(f"Время выполнения на python: {(middle_time-start_time):.6f} секунд")
#print(f"Посчитанный linegraph на python: {cent_p.eigenvector_centrality()[:5]}")
print(f"Время выполнения на c++: {(end_time-middle_time):.6f} секунд")
print(f"Посчитанный linegraph на c++: {cent_cpp.eigenvector_centrality()[:5]}")

Processing pairs:   0%|          | 0/20933685 [00:00<?, ?it/s]

Processing pairs: 24290000it [00:05, 4273381.26it/s]                              


Время выполнения на c++: 6.664248 секунд
Посчитанный linegraph на c++: [5.7906180384158486e-08, 5.79061803839531e-08, 1.2663303183219947e-07, 0.00047617013579704905, 0.10279805953419237]


In [21]:
import pickle

H_filtered = pickle.load(open("hypergraph_filtered.pkl", "rb"))

In [ ]:
ig = ultra_fast_line_vector_generator(H_filtered)

Processing pairs:   2%|▏         | 2609620000/150881999115 [09:06<5:33:16, 7414866.96it/s] 